In [ ]:
import json
import time
import pandas as pd
from datetime import datetime
from rpc.utility import load_json, save_json, strip_digt, to_vt_symbol

future_basic_data = pd.DataFrame()


def get_future_basic():
    """获取期货合约基础数据"""
    today = datetime.now()
    df = all_instruments(type="Future", market="cn", date=today)
    df["order_book_id"] = df["order_book_id"].map(strip_digt)
    df.drop_duplicates(subset="order_book_id", inplace=True)
    df.set_index("order_book_id", inplace=True)
    return df

    
def save_multiplier():
    """导出期货合约交易乘数到json文件"""
    df = future_basic_data
    d = dict(df["contract_multiplier"])
    save_json('multiplier.json', d)
    return d


def gf_dominant_lifespan(df):
    """
    分组处理函数
    按品种分组，获取每个主力合约的生命周期并返回为df
    """
    d = dict()
    d['start'] = df.index[0]
    d['end'] = df.index[-1]
    
    rq_symbol = df.iloc[0]['dominant']
    underlying = strip_digt(df.iloc[0]['dominant'])
    exchange = future_basic_data.loc[underlying]['exchange']
    vt_symbol = to_vt_symbol('.'.join([rq_symbol, exchange]))
    
    d['underlying'] = underlying
    d['exchange'] = exchange
    d['vt_symbol'] = vt_symbol
    new_df = pd.DataFrame([d])
    new_df = new_df[['start', 'end', 'underlying', 'exchange', 'vt_symbol']]
    return new_df


def get_dominant_start_end(underlying, end_date):
    """获取每个品种的主力合约起止日期数据"""
    s = futures.get_dominant(underlying, end_date=end_date)
    df = pd.DataFrame(s)

    df2 = df.groupby('dominant').apply(gf_dominant_lifespan)
    df2 = df2.droplevel(1)
    return df2


def save_all_dominant_period():
    commodities = future_basic_data.index.values
    end_date = datetime.now()
    dfs = []
    for commodity in commodities:
        dfs.append(get_dominant_start_end(commodity, end_date))
        print(f"{commodity}获取完成")
        time.sleep(1)
    df = pd.concat(dfs)
    df.to_csv('dominant_data.csv')
    
    print(f"{end_date}合并保存成功")
    return df
    

future_basic_data = get_future_basic()


In [ ]:
df = save_all_dominant_period()
df

In [ ]:
df = get_dominant_start_end('AP', datetime.now())
df

In [ ]:
d = save_multiplier()
d